In [2]:
from sumobot_real import Sumobot

import random, sys
import numpy as np
from keras import Sequential
from collections import deque
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam

env = Sumobot()
np.random.seed(0)


class DQN:

    """ Implementation of deep q learning algorithm """

    def __init__(self, action_space, state_space): #making the agent

        self.action_space = action_space
        self.state_space = state_space
        self.epsilon = 1
        self.gamma = .95
        self.batch_size = 64
        self.epsilon_min = .01
        self.epsilon_decay = .995
        self.learning_rate = 0.001
        self.memory = deque(maxlen=100000)
        self.model = self.build_model()
        self.episode_coords = [0,0] # will be a 2x2 matrix
        self.angle = 0
        self.radius = 44

    def build_model(self):
        
        model = Sequential() #groups a linear stack of layers into a model, provides training and inference features on this model
        model.add(Dense(64, input_shape=(self.state_space,), activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space) #choose a random move
        act_values = self.model.predict(state) # Generates output predictions for the input samples.
        return np.argmax(act_values[0]) 

    def replay(self):
    
#         print(len(self.memory))
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        states = np.squeeze(states)
        next_states = np.squeeze(next_states)

        targets = rewards + self.gamma*(np.amax(self.model.predict_on_batch(next_states), axis=1))*(1-dones)
        targets_full = self.model.predict_on_batch(states)

        ind = np.array([i for i in range(self.batch_size)])
        targets_full[[ind], [actions]] = targets

        self.model.fit(states, targets_full, epochs=1, verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def sumobot_spiral(self, angle, radius):
        x = math.sqrt((radius**2)/(1 + (math.tan(math.radians(angle)))**2))
        y = x * math.tan(math.radians(angle))
        if 0 <= angle <= 90:
            self.episode_coords[0] = [-x,-y]
        elif 90 < angle <= 180:
            self.episode_coords[0] = [x,y]
        elif 180 < angle <= 270:
            self.episode_coords[0] = [x,y]
        elif 270 < angle < 360:
            self.episode_coords[0] = [-x,-y]
            
    for i in range(3):
            self.sumobot.goto(self.radius, 0)
            while(self.angle < 360):             
                self.sumobot_spiral(self.angle, self.radius)
                self.enemy_locs(self.sumobot.xcor(), self.sumobot.ycor())
                self.angle += self.angle_delta
            self.angle -= 360 
            self.radius += 10 

        
    def enemy_locs(self, xcor, ycor):
        enemy_angle = 0
        dist = [17, 34, 51]
        enemy_angle_delta = 45
        ex = 0
        ey = 0
        for r in dist:  
            while(enemy_angle < 360):
                if enemy_angle <= 90 or enemy_angle > 270: 
                    ex = xcor - math.sqrt((r**2)/(1+(math.tan(math.radians(enemy_angle))**2)))
                    ey = ycor - (ex - xcor) * math.tan(math.radians(enemy_angle)) 
                else:
                    ex = xcor + math.sqrt((r**2)/(1+(math.tan(math.radians(enemy_angle))**2)))
                    ey = ycor + (ex - xcor) * math.tan(math.radians(enemy_angle))  
                    
                if (math.sqrt(ex**2 + ey**2) < self.arena_radius):
                    if 0 <= enemy_angle <= 90:
                        if enemy_angle == 90:
                            self.episode_coords[1] = [ex,-ey-r]
                        else:
                            self.episode_coords[1] = [ex,-ey]
                    elif 90 < enemy_angle <= 180:
                        if enemy_angle == 180:
                            self.episode_coords[1] = [ex-r,ey]
                        else:
                            self.episode_coords[1] = [ex,ey]
                    elif 180 < enemy_angle <= 270:
                        if enemy_angle == 270:
                            if r == 17 :
                                self.episode_coords[1] = [ex,ey+r]
                            else: 
                                self.episode_coords[1] = [ex,ey]
                        else:
                            self.episode_coords[1] = [ex,ey]
                    elif 270 < enemy_angle <= 360:
                        if enemy_angle == 315:
                            if r == 34:
                                self.episode_coords[1] = [ex,-ey]
                            else:
                                self.episode_coords[1] = [ex,-ey]
                        else:
                            self.episode_coords[1] = [ex,ey]
                else: 
                    pass
#                     self.enemy.pd()
#                     self.enemy.dot('red')
#                     self.enemy.pu()
                enemy_angle += enemy_angle_delta
#                 time.sleep(0.5)
            enemy_angle -= 360
            


def train_dqn(episode):

    loss = []
    #sumobot and enemy action and state space
    action_space = 9
    state_space = 4
    max_steps = 20
    #sumobot agent
    agent = DQN(action_space, state_space)
    for e in range(episode):
        # the reset here will take the four coordinates where to bot should move to
        # the only place where the algorithm resets the sumobot
        state = env.reset(self.episode_coords) 
        state = np.reshape(state, (1, state_space))
        score = 0
        print("episode: ", e)
        for i in range(max_steps):
            #enemey agent
            action = agent.act(state)
            reward, next_state, done = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            if done:
#                 print("episode: {}/{}, score: {}".format(e, episode, score))
                agent.model.summary()
                break
        loss.append(score)
    #save both models
    agent.model.save('model.h5')
    return loss


if __name__ == '__main__':
# So when the interpreter runs a module, the __name__ variable will be set as  __main__ if the module that is being run is the main program.
# But if the code is importing the module from another module, then the __name__  variable will be set to that module’s name.
# https://www.freecodecamp.org/news/if-name-main-python-example/
    ep = 100
    loss = train_dqn(ep)
#     print(loss)
    plt.plot([i for i in range(ep)], loss)
    plt.xlabel('episodes')
    plt.ylabel('reward')
    plt.show()

Connected
episode:  0


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/thebird/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/thebird/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/thebird/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 3452, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/Users/thebird/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 1779, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "/Users/thebird/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 1181, in loop
    rc = self.loop_read(max_packets)
  File "/Users/thebird/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 1572, in loop_read
    rc = self._packet_read()
  File "/Users/thebird/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 2310, in _packet_read
    rc

NameError: name 'angle' is not defined